**European Option Pricing**

We would be using the following ways: Black-Scholes Pricing, Pricing with Laplace Transform, Pricing with Monte Carlo. 
Firstly, we import main packages we will use:

In [2]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm
from scipy.integrate import quad 

Next, we input random parameters that we want to use, this can of course be changed accordingly. 

In [4]:
r = 0.02
sigma = 0.3
T = 1
K = 100
S0 = 120

**Black-Scholes Pricing**

Using Black Scholes model for pricing European Options
Using the formulas: 
$ V(t) = S(t) * cdf(d_1) - K e^{-r(T-t)} * cdf(d_2)$, where \
$ d_1 = \frac{\log\left(\frac{S(t)}{K}\right) + r(T-t) + \frac{1}{2} \sigma^2(T-t)}{\sigma \sqrt{(T-t)}}$, \
$ d_2 = \frac{\log\left(\frac{S(t)}{K}\right) + r(T-t) - \frac{1}{2} \sigma^2(T-t)}{\sigma \sqrt{(T-t)}}$

In [8]:
def european_BS(t, St, K, T, r, sigma, call):
    d1 = (np.log(St/K) + r * (T-t) + 0.5 * sigma**2 * (T-t)) / (sigma * np.sqrt(T-t))
    d2 = (np.log(St/K) + r * (T-t) - 0.5 * sigma**2 * (T-t)) / (sigma * np.sqrt(T-t))
    if call == 1:
        return St * norm.cdf(d1) - K * np.exp(-r*(T-t)) * norm.cdf(d2) 
    elif call == 0:
        return K * np.exp(-r * (T-t)) * norm.cdf(-d2) - St * norm.cdf(-d1)

In [10]:
V0_call = european_BS(0, S0, K, T, r, sigma, 1)
V0_put = european_BS(0, S0, K, T, r, sigma, 0)
V0_call, V0_put

(26.803371450525006, 4.8232387812005335)

**Pricing with Laplace transform**

We refer to the payoff with: $ f(x) = \int{f}(\tilde{z})e^{zx}dxV(0) = e^{-rT}\int{f}(\tilde{z})\chi(-iz)dz $ \

And using the bilateral laplace transform with: 
$ {f}(\tilde{z}) = \int{f}(\tilde{x})e^{-zx}dx= \frac{K^{1-z}}{z(z-1)} $, where f(x) is the payoff function of European option and with the characteristic function of \
$ \chi(u) = exp(iu(\log{S(0)}+rT)-(iu+u^{2})\frac{\sigma^{2}}{2}T) $ following the Black-Scholes formula. \
After deviation, we arrive at: $ V(0) = \frac{e^{-rT}}{\pi}\int_0^\infty{Re({f}(\tilde{R+1u})\chi(u - iR, S(0)))}du $

In [13]:
def charac_func(u, St, t, T, r, sigma):
    return np.exp(1j*u*(np.log(St) + r*(T-t)) - (1j*u+u**2) * sigma ** 2/2*(T-t))

In [15]:
f_tilde = lambda z: K**(1-z)/(z*(z-1))

In [17]:
def european_LP(St, t, T, r, sigma, R):
    intergrand = lambda u : np.real(f_tilde(R+1j*u)*charac_func(u-1j*R, St, t, T, r, sigma))
    return np.exp(-r*(T-t))/np.pi*quad(intergrand,0,np.inf)[0]

In [19]:
V0_call_LP = european_LP(S0, 0, T, r, sigma, R = 1.5)
V0_put_LP = european_LP(S0, 0, T, r, sigma, R = -0.5)
V0_call_LP, V0_put_LP

(26.803371450524807, 4.823238781200449)

**Pricing with Monte Carlo** 

$ V(0) = e^{-rT}E_Q({f}(x))$, where f(x) is the payoff function. \
Using Black-Scholes formula, we simulate N paths of Brownian Motion for S(T) and plug them in the pacoff function of interest. The option price is then the discounted sample average.

In [22]:
np.random.seed(10)
x_rand = np.random.normal(size = 10000)
S_T = S0 * np.exp((r - sigma**2 /2) * T + sigma * np.sqrt(T) * x_rand)
V0_call_MC = np.exp(-r*T) * np.mean(np.maximum(S_T - K, 0))
V0_put_MC = np.exp(-r*T) * np.mean(np.maximum(K - S_T, 0))
V0_call_MC, V0_put_MC

(26.76425551425964, 4.7259095999117395)